In [1]:
import math
import re
from typing import List, Dict, Any, Tuple
from llm_utils import LocalLLMClient, setup_logging

# ============== 配置 ==============
setup_logging(level="INFO")

# 创建全局本地LLM客户端（白盒模型）
client = LocalLLMClient(
    model_id="Qwen/Qwen3-4B-Thinking-2507",  # 本地模型
    local_path="./models/qwen3-4b-thinking",
    device="auto",  # 自动选择GPU/CPU
    max_tokens=200,
    temperature=0.7
)

# === 工具函数 ===
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def get_yes_no_probabilities_from_distribution(token_distribution: Dict[str, float]) -> Tuple[float, float]:
    """
    从token概率分布中提取Yes/No的概率
    """
    # 初始化概率
    yes_prob = 0.0
    no_prob = 0.0
    
    # 查找各种形式的Yes/No token
    yes_tokens = ["Yes", "yes", "YES", "是", "对的", "正确", "True", "true"]
    no_tokens = ["No", "no", "NO", "否", "不对", "错误", "False", "false"]
    
    # 累加所有Yes相关token的概率
    for token in yes_tokens:
        if token in token_distribution:
            yes_prob += token_distribution[token]
    
    # 累加所有No相关token的概率
    for token in no_tokens:
        if token in token_distribution:
            no_prob += token_distribution[token]
    
    # 如果没有找到任何Yes/No token，检查子串
    if yes_prob == 0 and no_prob == 0:
        for token, prob in token_distribution.items():
            token_lower = token.lower()
            if any(yes_word in token_lower for yes_word in ["yes", "是", "对", "true"]):
                yes_prob += prob
            elif any(no_word in token_lower for no_word in ["no", "否", "错", "false"]):
                no_prob += prob
    
    # 如果还是没有找到，尝试归一化第一个token的概率
    if yes_prob == 0 and no_prob == 0 and token_distribution:
        # 取前两个token的概率
        sorted_tokens = sorted(token_distribution.items(), key=lambda x: x[1], reverse=True)
        if len(sorted_tokens) >= 2:
            # 假设第一个token是"Yes"或"No"
            # 这里我们无法判断，返回0.5/0.5
            return 0.5, 0.5
    
    # 归一化概率
    total = yes_prob + no_prob
    if total > 0:
        yes_prob_normalized = yes_prob / total
        no_prob_normalized = no_prob / total
    else:
        yes_prob_normalized = 0.5
        no_prob_normalized = 0.5
    
    return yes_prob_normalized, no_prob_normalized

def llm_judge_with_token_prob(prompt, x1, x2, n_sample=5, method='token_prob'):
    """
    使用白盒LLM进行判断，基于token概率
    
    参数:
        prompt: 提示词
        x1, x2: 变量名
        n_sample: 采样次数（用于frequency模拟）
        method: 'token_prob' | 'frequency' | 'logit' (保持接口兼容性，实际都使用token概率)
    """
    if method == 'frequency':
        # 频率方法：多次采样，统计Yes/No出现频率
        votes = []
        yes_probs = []
        no_probs = []
        
        for _ in range(n_sample):
            try:
                # 调用本地模型，获取token概率
                response = client.chat(
                    prompt,
                    return_token_probs=True,
                    temperature=0.7  # 添加随机性
                )
                
                # 裁剪思考链
                cropped_response = response.crop_thinking()
                
                # 获取第一个token的概率分布
                token_dist = client.get_token_distributions(
                    cropped_response,
                    top_k=20,
                    skip_thinking=True,
                    skip_zeros=True,
                    zero_threshold=0.001
                )
                
                if len(token_dist) > 0:
                    # 从概率分布中提取Yes/No概率
                    yes_prob, no_prob = get_yes_no_probabilities_from_distribution(token_dist[0])
                    yes_probs.append(yes_prob)
                    no_probs.append(no_prob)
                    
                    # 根据概率投票
                    vote = 1 if yes_prob >= no_prob else 0
                    votes.append(vote)
            except Exception as e:
                print(f"采样失败: {e}")
                continue
        
        if not votes:
            p_yes = 0.5
            p_no = 0.5
        else:
            p_yes = sum(votes) / len(votes)
            p_no = 1 - p_yes
        
        # 计算平均概率
        avg_yes_prob = sum(yes_probs) / len(yes_probs) if yes_probs else p_yes
        avg_no_prob = sum(no_probs) / len(no_probs) if no_probs else p_no
        
        if p_yes >= p_no:
            return {"label": 1, "prob": avg_yes_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
        else:
            return {"label": 0, "prob": avg_no_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
    
    elif method == 'logit':
        # Logit方法：解析logits并计算概率
        try:
            response = client.chat(prompt, return_token_probs=True)
            text = response.content.strip()
            
            # 尝试从文本中解析logits
            matches = re.findall(r"([-]?[0-9]*\.?[0-9]+)", text)
            if len(matches) >= 2:
                l1, l2 = float(matches[0]), float(matches[1])
                logit_yes, logit_no = l1, l2
            else:
                # 如果无法解析，使用token概率
                return llm_judge_with_token_prob(prompt, x1, x2, n_sample=1, method='token_prob')
            
            p_yes = sigmoid(logit_yes)
            p_no = sigmoid(logit_no)
            
            # 归一化
            total = p_yes + p_no
            if total > 0:
                p_yes_normalized = p_yes / total
                p_no_normalized = p_no / total
            else:
                p_yes_normalized = 0.5
                p_no_normalized = 0.5
            
            if p_yes_normalized >= p_no_normalized:
                return {"label": 1, "prob": p_yes_normalized, "yes_prob": p_yes_normalized, "no_prob": p_no_normalized}
            else:
                return {"label": 0, "prob": p_no_normalized, "yes_prob": p_yes_normalized, "no_prob": p_no_normalized}
                
        except Exception as e:
            print(f"Logit方法失败: {e}, 回退到token_prob方法")
            return llm_judge_with_token_prob(prompt, x1, x2, n_sample=1, method='token_prob')
    
    else:  # token_prob 方法
        # 使用白盒模型的token概率
        all_yes_probs = []
        all_no_probs = []
        
        for i in range(max(1, n_sample)):  # 至少采样1次
            try:
                # 调用本地模型，获取token概率
                response = client.chat(
                    prompt,
                    return_token_probs=True,
                    temperature=0.1 if n_sample == 1 else 0.7  # 单次采样降低随机性
                )
                
                # 裁剪思考链
                cropped_response = response.crop_thinking()
                
                # 获取第一个token的概率分布
                token_dist = client.get_token_distributions(
                    cropped_response,
                    top_k=20,
                    skip_thinking=True,
                    skip_zeros=True,
                    zero_threshold=0.001
                )
                
                if len(token_dist) > 0:
                    # 从概率分布中提取Yes/No概率
                    yes_prob, no_prob = get_yes_no_probabilities_from_distribution(token_dist[0])
                    
                    all_yes_probs.append(yes_prob)
                    all_no_probs.append(no_prob)
                    
                    if n_sample > 1:
                        print(f"  采样{i+1}: P(Yes)={yes_prob:.4f}, P(No)={no_prob:.4f}")
                        
            except Exception as e:
                print(f"采样{i+1}失败: {e}")
                continue
        
        # 如果没有成功采样，使用默认值
        if not all_yes_probs:
            avg_yes_prob = 0.5
            avg_no_prob = 0.5
        else:
            # 计算平均概率
            avg_yes_prob = sum(all_yes_probs) / len(all_yes_probs)
            avg_no_prob = sum(all_no_probs) / len(all_no_probs)
        
        if n_sample > 1:
            print(f"  平均概率: P(Yes)={avg_yes_prob:.4f}, P(No)={avg_no_prob:.4f}")
        
        # 根据平均概率决定标签
        if avg_yes_prob >= avg_no_prob:
            return {"label": 1, "prob": avg_yes_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
        else:
            return {"label": 0, "prob": avg_no_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}

# === MoE 专家定义 ===
CAUSAL_EXPERTS = {
    "graph_theory": {
        "name": "因果图论专家",
        "description": "专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。推理方式：系统检查所有可能的路径，分析路径上的变量类型和连接方式，使用严谨的图论推理链条。",
        "specialty": "路径分析、环路检测、d-分离判断",
        "reasoning_style": "结构化图遍历",
        "output_format": "基于图结构的二值判断"
    },
    "statistical": {
        "name": "计量统计专家", 
        "description": "专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等统计概念。",
        "specialty": "独立性检验、相关性分析、混淆检测",
        "reasoning_style": "概率统计推理",
        "output_format": "基于统计证据的概率判断"
    },
    "domain_knowledge": {
        "name": "领域先验专家",
        "description": "基于现实世界知识和科学常识进行因果推理，考虑时间顺序、物理可能性、生物学机制等约束。推理方式：结合文献证据、科学理论和常识性约束。",
        "specialty": "机制分析、时序推理、现实约束",
        "reasoning_style": "基于证据的归纳推理", 
        "output_format": "基于领域知识的合理性判断"
    },
    "counterfactual": {
        "name": "反事实干预专家",
        "description": "从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。推理方式：构建反事实场景，分析干预后的可能变化。",
        "specialty": "干预分析、潜在结果、do算子",
        "reasoning_style": "反事实思维实验",
        "output_format": "基于干预推理的因果判断"
    },
    "temporal_dynamics": {
        "name": "时间动态专家",
        "description": "专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈。推理方式：严格检查时间顺序，分析因果链的时间特性。",
        "specialty": "时序分析、动态过程、延迟效应",
        "reasoning_style": "时间序列推理", 
        "output_format": "基于时间顺序的因果判断"
    },
    "mechanism_modeling": {
        "name": "机制建模专家", 
        "description": "专注于因果机制的可解释性建模，分析中间变量、中介效应和机制路径。推理方式：构建机制框图，分析变量间的功能关系。",
        "specialty": "中介分析、机制路径、功能关系",
        "reasoning_style": "机制分解建模",
        "output_format": "基于机制完整性的判断"
    },
    "robustness_analysis": {
        "name": "稳健性检验专家",
        "description": "专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。推理方式：进行敏感性分析，检验边界条件和假设变化的影响。",
        "specialty": "敏感性分析、稳健检验、边界情况",
        "reasoning_style": "多情景验证",
        "output_format": "基于稳健性评估的判断"
    }
}

# === Router 函数（增强版）===
def expert_router(question_type: str, x1: str, x2: str) -> List[str]:
    """
    根据问题类型和变量特征选择最相关的专家
    """
    # 基础路由规则
    routing_rules = {
        "backdoor_path": [
            "graph_theory", "statistical", "counterfactual", "temporal_dynamics", 
            "mechanism_modeling", "robustness_analysis", "domain_knowledge"
        ],
        "independence": [
            "statistical", "graph_theory", "counterfactual", "robustness_analysis",
            "temporal_dynamics", "mechanism_modeling", "domain_knowledge"
        ],
        "latent_confounder": [
            "domain_knowledge", "statistical", "mechanism_modeling", "counterfactual",
            "robustness_analysis", "graph_theory", "temporal_dynamics"
        ],
        "causal_direction": [
            "temporal_dynamics", "domain_knowledge", "counterfactual", "mechanism_modeling",
            "statistical", "graph_theory", "robustness_analysis"
        ]
    }
    
    # 获取基础专家列表
    base_experts = routing_rules.get(question_type, list(CAUSAL_EXPERTS.keys()))
    
    # 使用门诊LLM agent来智能选择专家
    try:
        clinic_recommendation = clinic_agent_recommend(question_type, x1, x2, base_experts)
        return clinic_recommendation
    except Exception as e:
        print(f"门诊agent路由失败: {e}，使用基础路由")
        # 失败时返回基础专家列表的前3个
        return base_experts[:3]

def clinic_agent_recommend(question_type: str, x1: str, x2: str, base_experts: List[str]) -> List[str]:
    """
    门诊LLM agent：根据具体变量和问题类型推荐最相关的专家
    """
    # 使用独立的LLM客户端进行推荐
    clinic_client = LocalLLMClient(
        model_id="Qwen/Qwen3-4B-Thinking-2507",
        local_path="./models/qwen3-4b-thinking",
        device="auto",
        max_tokens=200,
        temperature=0.1
    )
    
    # 构建专家选择提示
    experts_description = "\n".join([
        f"- {expert}: {CAUSAL_EXPERTS[expert]['description']}" 
        for expert in base_experts
    ])
    
    clinic_prompt = f"""
作为因果推断门诊专家，你需要为以下因果分析任务选择最合适的专家组合：

**分析任务**: {question_type}
**变量对**: {x1} 和 {x2}

**可用专家列表**:
{experts_description}

**选择要求**:
1. 根据变量内容和问题类型，选择最相关的3个专家
2. 按相关性从高到低排序
3. 确保专家视角的多样性（不要选择推理方式相似的专家）
4. 考虑变量的领域特性（医学、经济、社会等）

请按照以下格式输出：
最终推荐专家: 专家1, 专家2, 专家3

**注意**: 只输出专家名称，用逗号分隔，不要添加其他文字。
"""
    
    # 调用LLM获取推荐
    response = clinic_client.chat(clinic_prompt)
    response_text = response.content.strip()
    print(f"门诊agent原始响应: {response_text}")
    
    # 解析返回的专家列表
    recommended_experts = parse_clinic_recommendation(response_text, base_experts)
    
    print(f"门诊agent推荐: {recommended_experts}")
    
    return recommended_experts

def parse_clinic_recommendation(response_text: str, base_experts: List[str]) -> List[str]:
    """
    解析门诊agent的推荐结果
    """
    # 方法1: 查找"最终推荐专家"后的内容
    if "最终推荐专家" in response_text:
        parts = response_text.split("最终推荐专家")
        if len(parts) > 1:
            expert_line = parts[1].strip().lstrip(":").strip()
            return extract_experts_from_line(expert_line, base_experts)
    
    # 方法2: 查找最后一行
    lines = [line.strip() for line in response_text.split('\n') if line.strip()]
    if lines:
        last_line = lines[-1]
        experts = extract_experts_from_line(last_line, base_experts)
        if len(experts) >= 2:
            return experts
    
    # 方法3: 在整个文本中搜索专家名称
    found_experts = []
    for expert in base_experts:
        if expert in response_text:
            found_experts.append(expert)
    
    if len(found_experts) >= 2:
        return found_experts[:3]  # 取前3个找到的专家
    
    # 如果所有方法都失败，返回基础专家前3个
    print(f"门诊agent解析不充分，使用基础专家: {base_experts[:3]}")
    return base_experts[:3]

def extract_experts_from_line(line: str, base_experts: List[str]) -> List[str]:
    """从一行文本中提取专家名称"""
    experts = []
    
    # 清理行内容
    clean_line = line.replace('：', ':').replace('，', ',').replace(' ', '')
    
    # 多种分割方式尝试
    separators = [',', '、', ';', '，']
    
    for sep in separators:
        if sep in clean_line:
            parts = [part.strip() for part in clean_line.split(sep)]
            break
    else:
        parts = [clean_line]
    
    for part in parts:
        clean_part = part.lower().replace('专家', '').replace('expert', '').strip()
        
        # 直接匹配专家名称
        for expert in base_experts:
            if (expert in clean_part or 
                expert.replace('_', ' ') in clean_part or
                CAUSAL_EXPERTS[expert]['name'] in part):
                if expert not in experts:
                    experts.append(expert)
                    break
        
        if len(experts) >= 3:
            break
    
    return experts

# === 专家提示创建函数 ===
def create_expert_prompt(base_prompt: str, expert_type: str, x1: str, x2: str) -> str:
    """
    为不同专家创建专业化的prompt
    """
    expert_info = CAUSAL_EXPERTS[expert_type]
    
    expert_specific_prompts = {
        "graph_theory": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于图结构的二值判断

分析步骤：
1. 构建因果图模型，识别所有可能的路径
2. 应用d-分离准则分析路径阻塞情况  
3. 检查后门路径、前门路径和混杂路径
4. 基于图结构做出明确的二值判断

请严格按照图论原理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "statistical": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于统计证据的二值判断

分析步骤：
1. 评估变量间的统计相关性
2. 考虑条件独立性和混淆因素
3. 分析统计显著性和置信度
4. 基于概率证据做出明确的二值判断

请基于统计原理进行严谨分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "domain_knowledge": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于领域知识的二值判断

分析步骤：
1. 调用相关领域的科学知识和常识
2. 考虑物理/生物/社会机制的合理性
3. 评估时间顺序和现实约束条件
4. 基于先验知识做出明确的二值判断

请结合现实世界知识进行推理，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "counterfactual": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于干预推理的二值判断

分析步骤：
1. 构建干预场景（do-操作）
2. 比较实际结果与反事实结果
3. 分析潜在结果分布
4. 基于干预效应做出明确的二值判断

请使用反事实推理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "temporal_dynamics": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于时间顺序的二值判断

分析步骤：
1. 严格检查原因和结果的时间顺序
2. 分析延迟效应和动态过程
3. 考虑时间序列的因果结构
4. 基于时间约束做出明确的二值判断

请重点分析时间维度，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "mechanism_modeling": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于机制完整性的二值判断

分析步骤：
1. 识别可能的中间机制和中介变量
2. 分析因果链的功能完整性
3. 评估机制路径的合理性
4. 基于机制可解释性做出明确的二值判断

请专注于机制分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "robustness_analysis": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于稳健性评估的二值判断

分析步骤：
1. 测试不同假设条件下的结果稳定性
2. 进行敏感性分析和边界检验
3. 评估结论的稳健程度
4. 基于稳健性评估做出明确的二值判断

请重点分析结论的可靠性，直接输出是或否（Yes/No）。\n\n{base_prompt}"""
    }
    
    return expert_specific_prompts.get(expert_type, base_prompt)

# === MoE 集成函数 ===
def aggregate_expert_judgments(expert_results: List[Dict]) -> Dict:
    """
    整合多个专家的判断结果
    """
    if not expert_results:
        return {"label": 0, "prob": 0.5}
    
    # 简单加权平均
    total_prob_yes = 0
    total_weight = 0
    
    for result in expert_results:
        weight = result.get("confidence", 1.0)
        # 使用yes_prob字段（从token概率得来）
        prob_yes = result.get("yes_prob", result["prob"] if result["label"] == 1 else 1 - result["prob"])
        total_prob_yes += prob_yes * weight
        total_weight += weight
    
    aggregated_prob_yes = total_prob_yes / total_weight if total_weight > 0 else 0.5
    
    if aggregated_prob_yes >= 0.5:
        return {"label": 1, "prob": aggregated_prob_yes, "yes_prob": aggregated_prob_yes, "no_prob": 1 - aggregated_prob_yes}
    else:
        return {"label": 0, "prob": 1 - aggregated_prob_yes, "yes_prob": aggregated_prob_yes, "no_prob": 1 - aggregated_prob_yes}

# === MoE判断函数 ===
def run_step_with_moe(base_prompt: str, x1: str, x2: str, question_type: str, method: str = 'token_prob') -> Dict:
    """
    使用MoE架构运行单个判断步骤
    """
    # 1. 路由选择专家
    selected_experts = expert_router(question_type, x1, x2)
    print(f"为问题 '{question_type}' 选择的专家: {selected_experts}")
    
    # 2. 执行专家判断
    expert_results = []
    for expert in selected_experts:
        expert_prompt = create_expert_prompt(base_prompt, expert, x1, x2)
        try:
            # 使用本地白盒模型的token概率方法
            result = llm_judge_with_token_prob(expert_prompt, x1, x2, n_sample=3, method=method)
            result["expert"] = expert
            result["confidence"] = 1.0
            expert_results.append(result)
            print(f"专家 {expert} 判断完成: label={result['label']}, P(Yes)={result.get('yes_prob', result['prob']):.4f}, P(No)={result.get('no_prob', 1-result['prob']):.4f}")
        except Exception as e:
            print(f"专家 {expert} 执行失败: {e}")
            continue
    
    # 3. 如果没有专家成功，使用默认方法
    if not expert_results:
        print("所有专家执行失败，使用默认方法")
        return llm_judge_with_token_prob(base_prompt, x1, x2, n_sample=5, method=method)
    
    # 4. 整合专家意见
    final_result = aggregate_expert_judgments(expert_results)
    final_result["expert_results"] = expert_results
    
    print(f"专家整合结果: label={final_result['label']}, P(Yes)={final_result['yes_prob']:.4f}, P(No)={final_result['no_prob']:.4f}")
    return final_result

# === 具体判断函数 ===
def check_backdoor(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断在这些变量中，变量 {x1} 和 {x2} 之间是否存在 back-door path（后门路径）。

后门路径是指从 {x1} 到 {x2} 的路径，其中包含指向 {x1} 的箭头，且这条路径没有被阻断。

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "backdoor_path", method)

def check_independence_after_block(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

如果阻断了 {x1} 和 {x2} 之间的所有 back-door path，那么 {x1} 与 {x2} 是否条件独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method)

def check_latent_confounder_after_block(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，是否仍然存在未观察到的潜在混杂因子同时影响 {x1} 和 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method)

def check_causal_direction_after_block(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method)

def check_independence(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 是否独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method)

def check_latent_confounder(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 之间是否存在未观察到的潜在混杂因子？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method)

def check_causal_direction(x1, x2, all_variables, method='token_prob'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method)

# === 完整的树查询函数 ===
def tree_query(x1, x2, all_variables, method='token_prob'):
    """
    基于树状逻辑的因果方向查询器（使用白盒token概率）
    """
    log = []

    # Step 1: 是否存在 backdoor path?
    print("=== Step 1: 检查后门路径 ===")
    res_backdoor = check_backdoor(x1, x2, all_variables, method)
    log.append(("backdoor_path", res_backdoor))

    if res_backdoor["label"] == 1:
        print("存在后门路径，进入阻断后分析路径")
        # Step 2: 阻断路径后是否独立？
        print("=== Step 2: 阻断后检查独立性 ===")
        res_ind = check_independence_after_block(x1, x2, all_variables, method)
        log.append(("independent_after_block", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["yes_prob"], "log": log}

        # Step 3: 是否存在潜在混杂因子？
        print("=== Step 3: 检查潜在混杂因子 ===")
        res_latent = check_latent_confounder_after_block(x1, x2, all_variables, method)
        log.append(("latent_confounder_after_block", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["yes_prob"], "log": log}

        # Step 4: 判断方向 (x→y?)
        print("=== Step 4: 判断因果方向 ===")
        res_dir = check_causal_direction_after_block(x1, x2, all_variables, method)
        log.append(("x->y_after_block", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["yes_prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["no_prob"], "log": log}

    else:
        print("不存在后门路径，进入直接分析路径")
        # 不存在 backdoor path
        res_ind = check_independence(x1, x2, all_variables, method)
        log.append(("independent_no_backdoor", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["yes_prob"], "log": log}

        res_latent = check_latent_confounder(x1, x2, all_variables, method)
        log.append(("latent_confounder_no_backdoor", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["yes_prob"], "log": log}

        res_dir = check_causal_direction(x1, x2, all_variables, method)
        log.append(("x->y_no_backdoor", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["yes_prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["no_prob"], "log": log}

# === 使用示例 ===
if __name__ == "__main__":
    # 定义完整的变量集合
    all_variables = ["冰淇淋销量", "溺水人数", "温度"]
    
    print("开始因果推断分析（使用本地白盒模型 + token概率）")
    print("=" * 60)
    
    # 测试不同的方法
    methods = ['token_prob', 'frequency', 'logit']
    all_results = {}
    
    for method in methods:
        print(f"\n使用方法: {method}")
        print("-" * 40)
        
        result = tree_query("冰淇淋销量", "溺水人数", all_variables, method=method)
        all_results[method] = result
        
        print(f"\n=== {method}方法结果 ===")
        print(f"变量关系: {result['relation']}")
        print(f"置信度 (P(Yes)): {result['confidence']:.4f}")
        
        # 保存详细日志
        print(f"\n=== 详细执行日志 ===")
        for step_name, step_result in result["log"]:
            print(f"\n{step_name}:")
            print(f"  最终判断: {'Yes' if step_result['label'] == 1 else 'No'}")
            print(f"  P(Yes): {step_result.get('yes_prob', step_result['prob']):.4f}")
            print(f"  P(No): {step_result.get('no_prob', 1-step_result['prob']):.4f}")
            
            if "expert_results" in step_result:
                print(f"  专家详情:")
                for expert_result in step_result["expert_results"]:
                    expert_name = expert_result.get("expert", "unknown")
                    yes_prob = expert_result.get("yes_prob", expert_result["prob"] if expert_result["label"] == 1 else 1 - expert_result["prob"])
                    no_prob = expert_result.get("no_prob", 1 - yes_prob)
                    print(f"    - {expert_name}: {'Yes' if expert_result['label'] == 1 else 'No'} (P(Yes)={yes_prob:.4f}, P(No)={no_prob:.4f})")
    
    # 比较不同方法的结果
    print(f"\n{'='*60}")
    print("=== 不同方法结果比较 ===")
    for method, result in all_results.items():
        print(f"{method}: 关系={result['relation']}, 置信度={result['confidence']:.4f}")
    
    print("\n" + "=" * 60)
    
    # 清理模型
    client.unload_model()
    print("模型已卸载")

12-09 17:42:01 [INFO] llm_utils.local_client - Model validation successful: ./models/qwen3-4b-thinking
12-09 17:42:01 [INFO] llm_utils.local_client - Model found and validated at local path: ./models/qwen3-4b-thinking
12-09 17:42:03 [INFO] llm_utils.local_client - Loading tokenizer from: ./models/qwen3-4b-thinking
12-09 17:42:04 [INFO] llm_utils.local_client - Loading model from: ./models/qwen3-4b-thinking
12-09 17:42:04 [INFO] llm_utils.local_client - Device: auto, Dtype: torch.float16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:42:07 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
开始因果推断分析（使用本地白盒模型 + token概率）

使用方法: token_prob
----------------------------------------
=== Step 1: 检查后门路径 ===
12-09 17:42:07 [INFO] llm_utils.local_client - Model validation successful: ./models/qwen3-4b-thinking
12-09 17:42:07 [INFO] llm_utils.local_client - Model found and validated at local path: ./models/qwen3-4b-thinking
12-09 17:42:07 [INFO] llm_utils.local_client - Loading tokenizer from: ./models/qwen3-4b-thinking
12-09 17:42:07 [INFO] llm_utils.local_client - Loading model from: ./models/qwen3-4b-thinking
12-09 17:42:07 [INFO] llm_utils.local_client - Device: auto, Dtype: torch.float16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:42:11 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：冰淇淋销量 和 溺水人数。分析类型是 backdoor_path，这在因果推断中指的是后门路径，用于处理混淆变量。

我需要从可用专家列表中选择最相关的3个专家，按相关性从高到低排序。专家列表包括：
- graph_theory
- statistical
- counterfactual
- temporal_dynamics
- mechanism_modeling
- robustness_analysis
- domain_knowledge

选择要求：
1. 根据变量内容和问题类型：变量是冰淇淋销量（冰淇淋销售）和溺水人数（溺水事件）。问题类型是 backdoor_path，这涉及因果图中的后门准则，用于识别和控制混淆变量。
2. 按相关性从高到低排序。
3. 确保专家视角的多样性：不要选择推理方式相似的专家。
4. 考虑变量的领域特性：冰淇淋销量和
门诊agent推荐: ['graph_theory', 'statistical', 'counterfactual']
为问题 'backdoor_path' 选择的专家: ['graph_theory', 'statistical', 'counterfactual']
  采样1: P(Yes)=0.5000, P(No)=0.5000
  采样2: P(Yes)=0.5000, P(No)=0.5000
  采样3: P(Yes)=0.5000, P(No)=0.5000
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
  采样1: P(Yes)=0.5000, P(No)=0.5000
  采样2: P(Yes)=0.5000, P(No)=0.5000
  采样3: P(Yes)=0.5000, P(No)=0.5000
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 s

...

12-09 17:49:14 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：冰淇淋销量 和 溺水人数。问题类型是 "independence"，意思是独立性分析。我需要选择最合适的3个专家组合，从给定的列表中。

回顾专家列表：

- statistical: 专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等。

- graph_theory: 专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。

- counterfactual: 从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。

- robustness_analysis: 专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。

- temporal_dynamics: 专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈
门诊agent推荐: ['statistical', 'graph_theory', 'counterfactual']
为问题 'independence' 选择的专家: ['statistical', 'graph_theory', 'counterfactual']
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 counterfactual 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家整合结果: label=1, P(Yes)=0.5000, P(No)=0.5000

=== frequency方法结果 ===
变量关系: independent
置信度 (P(Yes)): 0.5000

=== 详细执行日志 ===

backdoor_path:
  最终判断: Yes
  P(Yes): 0.5000
 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:51:30 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：冰淇淋销量 和 溺水人数。分析类型是 backdoor_path，这在因果推断中指的是后门路径（backdoor path）的分析，用于处理混淆变量。

关键点是：冰淇淋销量和溺水人数。我需要回想一下，冰淇淋销量和溺水人数之间有什么因果关系？在现实中，冰淇淋销量和溺水人数通常有正相关，因为夏天热的时候，人们吃更多冰淇淋，同时游泳更多，溺水事件也增多。但这里没有直接的因果；它们可能都受天气（如温度）的影响。所以，天气是混淆变量。

在因果推断中，后门路径分析用于当存在一个共同原因（混淆变量）时，如何估计因果效应。例如，想从冰淇淋销量（原因）到溺水人数（结果）的因果效应，但需要控制混淆变量。

现在，我需要从可用专家列表中选择最相关的3个
门诊agent解析不充分，使用基础专家: ['graph_theory', 'statistical', 'counterfactual']
门诊agent推荐: ['graph_theory', 'statistical', 'counterfactual']
为问题 'backdoor_path' 选择的专家: ['graph_theory', 'statistical', 'counterfactual']
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 counterfactual 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家整合结果: label=1, P(Yes)=0.5000, P(No)=0.5000
存在后门路径，进入阻断后分析路径
=== Step 2: 阻断后检查独立性 ===
12-09 17:53:07 [INFO] llm_utils.local_client - Model validation successful

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:53:11 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：冰淇淋销量 和 溺水人数。问题类型是 "independence"，意思是独立性分析。我需要选择最合适的3个专家组合，从给定的列表中。

可用专家列表：
- statistical: 专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等。
- graph_theory: 专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。
- counterfactual: 从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。
- robustness_analysis: 专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。
- temporal_dynamics: 专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈
门诊agent推荐: ['statistical', 'graph_theory', 'counterfactual']
为问题 'independence' 选择的专家: ['statistical', 'graph_theory', 'counterfactual']
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家 counterfactual 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
专家整合结果: label=1, P(Yes)=0.5000, P(No)=0.5000

=== logit方法结果 ===
变量关系: independent
置信度 (P(Yes)): 0.5000

=== 详细执行日志 ===

backdoor_path:
  最终判断: Yes
  P(Yes): 0.5000
  P(No): 0

In [2]:
from itertools import combinations

def compute_all_causal_relations(variables, method='probability'):
    """
    计算图中每两个变量之间的因果关系，使用tree_query函数。
    
    输出:
        {
            (x1, x2): {
                'relation': 'x->y' | 'y->x' | 'x<->y' | 'independent',
                'confidence': float,
                'log': [(step_name, {'label': int, 'prob': float}), ...]
            },
            ...
        }
    """
    all_relations = {}

    # 生成所有变量的组合 C(n, 2)
    for x1, x2 in combinations(variables, 2):
        # 进行 tree_query
        result = tree_query(x1, x2, method)
        
        # 存储结果
        all_relations[(x1, x2)] = result

    return all_relations


In [3]:
variables = ['气温', '冰淇淋销量', '溺水人数']
relations = compute_all_causal_relations(variables)

for (x1, x2), relation in relations.items():
    print(f"Relation between {x1} and {x2}: {relation['relation']} (Confidence: {relation['confidence']})")


=== Step 1: 检查后门路径 ===
12-09 17:54:43 [INFO] llm_utils.local_client - Model validation successful: ./models/qwen3-4b-thinking
12-09 17:54:43 [INFO] llm_utils.local_client - Model found and validated at local path: ./models/qwen3-4b-thinking
12-09 17:54:43 [INFO] llm_utils.local_client - Loading tokenizer from: ./models/qwen3-4b-thinking
12-09 17:54:44 [INFO] llm_utils.local_client - Loading model from: ./models/qwen3-4b-thinking
12-09 17:54:44 [INFO] llm_utils.local_client - Device: auto, Dtype: torch.float16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:54:47 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：气温 和 冰淇淋销量。具体是backdoor_path，这在因果推断中指的是后门路径，用于处理混淆变量。

我需要从可用专家列表中选择最相关的3个专家，按相关性从高到低排序。专家列表包括：
- graph_theory
- statistical
- counterfactual
- temporal_dynamics
- mechanism_modeling
- robustness_analysis
- domain_knowledge

选择要求：
1. 根据变量内容和问题类型：气温和冰淇淋销量。这是一个经济或社会领域的问题，涉及天气和消费行为。气温是原因，冰淇淋销量是结果。在现实中，气温升高可能导致冰淇淋销量增加，但可能有混淆变量，比如季节变化（夏季）或经济状况。
2. 问题类型是backdoor_path：这在因果推断中，后门路径用于当存在混淆变量时，如何正确估计因果效应。例如，
门诊agent推荐: ['graph_theory', 'statistical', 'counterfactual']
为问题 'backdoor_path' 选择的专家: ['graph_theory', 'statistical', 'counterfactual']
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() first.
采样3失败: Model not initialized. Call _load_model() first.
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:55:02 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：气温 和 冰淇淋销量。问题类型是 "independence"，意思是独立性分析。我需要选择最合适的3个专家组合，从给定的列表中。

可用专家列表：
- statistical: 专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等。
- graph_theory: 专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。推理方式：系统检查所有可能的路径，分析路径上的变量类型和连接方式。
- counterfactual: 从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。推理方式：构建反事实场景，分析干预后的可能变化。
- robustness_analysis: 专门评估因果关系的稳健性和敏感性，考虑不同
门诊agent推荐: ['statistical', 'graph_theory', 'counterfactual']
为问题 'independence' 选择的专家: ['statistical', 'graph_theory', 'counterfactual']
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() first.
采样3失败: Model not initialized. Call _load_model() first.
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() f

...

12-09 17:55:32 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：气温 和 溺水人数。问题类型是 "independence"，意思是独立性分析。我需要选择最合适的3个专家组合，从给定的列表中。

可用专家列表：
- statistical: 专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等。
- graph_theory: 专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。
- counterfactual: 从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。
- robustness_analysis: 专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。
- temporal_dynamics: 专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈。
门诊agent推荐: ['statistical', 'graph_theory', 'counterfactual']
为问题 'independence' 选择的专家: ['statistical', 'graph_theory', 'counterfactual']
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() first.
采样3失败: Model not initialized. Call _load_model() first.
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:55:46 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，任务是分析变量对：冰淇淋销量 和 溺水人数。分析类型是 backdoor_path，这在因果推断中指的是后门路径，用于处理混淆变量。

我需要从可用专家列表中选择最相关的3个专家，按相关性从高到低排序。专家列表包括：
- graph_theory
- statistical
- counterfactual
- temporal_dynamics
- mechanism_modeling
- robustness_analysis
- domain_knowledge

选择要求：
1. 根据变量内容和问题类型，选择最相关的3个专家。
2. 按相关性从高到低排序。
3. 确保专家视角的多样性（不要选择推理方式相似的专家）。
4. 考虑变量的领域特性：冰淇淋销量和溺水人数属于社会/经济领域，可能涉及天气、季节等。

变量内容：冰淇淋销量（冰淇淋销售量）和溺水人数（溺
门诊agent推荐: ['graph_theory', 'statistical', 'counterfactual']
为问题 'backdoor_path' 选择的专家: ['graph_theory', 'statistical', 'counterfactual']
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() first.
采样3失败: Model not initialized. Call _load_model() first.
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 graph_theory 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

12-09 17:56:01 [INFO] llm_utils.local_client - Local LLM client initialized: Qwen/Qwen3-4B-Thinking-2507
门诊agent原始响应: 首先，分析任务是"independence"，意思是独立性。变量对是"冰淇淋销量"和"溺水人数"。我需要为这个任务选择最合适的3个专家组合。

关键点是：任务是关于独立性的。在因果推断中，独立性通常指两个变量是否独立，即是否有因果关系或相关性。但这里指定是"independence"，所以我应该关注统计独立性或条件独立性。

变量：冰淇淋销量（ice cream sales）和溺水人数（drowning incidents）。这是一个经典的例子，常被用来说明相关性不等于因果性。例如，冰淇淋销量和溺水人数在夏季都高，但冰淇淋销量不是导致溺水的原因；它们都受温度影响（热天导致更多冰淇淋消费和更多游泳，从而溺水）。

在因果推断中，独立性分析可能涉及检查这两个变量是否独立，或者在给定某些变量（如温度）下是否独立。
门诊agent解析不充分，使用基础专家: ['statistical', 'graph_theory', 'counterfactual']
门诊agent推荐: ['statistical', 'graph_theory', 'counterfactual']
为问题 'independence' 选择的专家: ['statistical', 'graph_theory', 'counterfactual']
采样1失败: Model not initialized. Call _load_model() first.
采样2失败: Model not initialized. Call _load_model() first.
采样3失败: Model not initialized. Call _load_model() first.
  平均概率: P(Yes)=0.5000, P(No)=0.5000
专家 statistical 判断完成: label=1, P(Yes)=0.5000, P(No)=0.5000
采样1失败: Model not initialized. Call _load_model() firs